<img src="images\website_logo_transparent_background.png" style="float: center; height: 100px; width: 100px;"></img>

<h2 style="align: center;">MO !</h2>

<p>Business intelligence, and data science, are integral positions. Analytics groups have the ability to work across business units to develop a unified understanding of: the internal operating environment, employee activity and engagement, and client activity (etc.). By harvesting this data, and reproducing it in a digestable and presentable format, the analytics function has the power to generate impactful insights that can educate teammates, accelerate sales, refine marketing activity, and, ultimately, generate revenue and profitability. 

This notebook contains alterable open-source code for automating tedious tasks: data cleaning, integrity, table transformation, and visualization. 

MO is: an ideal foundation for your AI projects and data science voyage; an introduction to operationalizing EDA (exploratory data analysis) with Python; a simple script that can do some heavy lifting. If you're keen, build on this code, make statements and loops resource-efficient, create a front-end interface, implement more advanced statistical methods - there are endless possibilities.

While the script was designed to be run from the command-line, or powershell, the notebook allows us to highlight essential elements, and provide thorough documentation. The intention was to create a standard pipeline for any dataset, however, some of the code below is tailored to the test data. Recompile on your machine, and alter accordingly. 

Thorough analysis is a vital responsibility, so, hopefully you will find this helpful. 

This code is NOT my own - I explored a variety of resources to compile a script that worked for analytical tasks.</p>

<p> If find this useful, please consider becoming a Patreon to support future efforts!</p>

[Become a Patreon]("https://www.patreon.com/think_elucidate")

----------------------------------------------------------------------------------------------------------------------------

<p>*Working knowledge of Python is required.</p>
<p>*Expect debugging.</p>
<p>*This code was written on a Windows operating system.</p>

----------------------------------------------------------------------------------------------------------------------------

In [ ]:
# FIRST IMPORT THE NECESSARY MODULES AND PACKAGES.
from ipypublish import nb_setup
from IPython.display import display, HTML

import pandas as pd
import pandas_profiling
import sklearn as skl
import numpy as np
import pprint as pp
import os, sys, shutil, pathlib
from pathlib import Path
import time
#import sqlite3
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
import json
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats
import pprint as pp

# PLOTLY IS MY PREFERRED VISUALIZATION LIBRARY, SO WE WILL IMPORT THOSE MODULES
# AS WELL. 
import plotly
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py

Depending on the task at hand, automatically reading a file from a specified directory may not be necessary. However, creating a designated workspace is, so,

The code below allows you to:
    - Set the working directory.
    - Set the path to the working file.
    - Read the file into memory based on the file extension.
    
For illustrative purposes, these lines are commented out, and we will read the test data from the github repository.

In [ ]:
# LET'S SET THE WORKING-DIRECTORY / WORKSPACE

#print("The current working directory is: ", os.getcwd(), '\n')
# you can enter the path without quotations or // separation
#path = input("Please enter the path to the desired directory: ")
#os.chdir(path)
#file_location = input("Please enter the file location: ")
#file_location = str(file_location)

In [ ]:
# READ THE DATA INTO MEMORY DEPENDING ON THE FILE EXTENSION
# --------------------------------------------------------------------------
# BASED ON MY EXPERIENCE, THE MAJORITY WILL BE READING FROM EXCEL WORKBOOKS,
# SO ALTERNATE METHODS ARE COMMENTED OUT, AND EXCLUDED.
# ---------------------------------------------------------------------------

#file_name, file_extension = os.path.splitext(file_location)
#if file_extension == '.csv':
#    data = pd.read_csv(file_location, encoding = 'iso-8859-1')
#elif file_extension == '.xls' or '.xlsx':
#    data = pd.read_excel(file_location)

#tree = ET.parse(file_location)
#root = tree.getroot()

#con = sqlite3.connect(file_location)
#data = pd.read_sql_query("SELECT * FROM Data;", con)
#con.close()

data = pd.read_csv("test_data\who-covid-situation-reports.csv")

print('\n', '~~~ File Loaded !! ~~~', '\n')

Exploratory Data Analysis (EDA) is critical to understand what you are working with on a deeper level. 

Pandas Profile Report is an exemplary module that provides you with a comprehensive report of variables, data types, null values, as well as an interaction visual and correlation matrix.

In [ ]:
# ------------------------------------------------------------------------
# PROCESSING AND DUE DILIGENCE
# ------------------------------------------------------------------------
profile = data.profile_report(title='Pandas Profile Report')
profile.to_notebook_iframe()

# THE PROFILE CAN ALSO BE SAVED AS AN INTERACTIVE HTML FILE
profile.to_file("profile.html")
        
df_length = data.shape[0]

# AFTER EXPLORING THE DATA, WE CAN HANDLE SOME OF THE QUESTION MARKS
nulls = data.isnull().sum()
nulls = nulls.to_dict()
to_drop = []

for k, v in nulls.items():
    # THE STATEMENT BELOW CAN BE ALTERED TO (v/df_length >= 0.8), OR AN ALTERNATE LOGICAL THRESHOLD
    if (v == df_length):
        to_drop.append(k)
data.drop(to_drop, axis=1, inplace=True)
data.drop_duplicates(keep='first', inplace=True)

print("AFTER DROPPING EGREGIOUSLY NULL COLUMNS, AND HANDLING DUPLICATE OBSERVATIONS; THE SHAPE OF THE DATAFRAME IS:")   
display(data.shape)
print('\n')

In [ ]:
# -------------------------------------------------------
# SUBSETTING THE MAIN DATAFRAME BY VARIABLES' DATA TYPES
# -------------------------------------------------------

# INITIALIZING POSITIONAL INDEXES FOR THE DATAFRAME
shape = data.shape
obs_num = int(shape[0])
var_num = int(shape[1])
display(data.dtypes)
print('\n')

# CREATING DTYPE SUBSETS
numerics = data.select_dtypes(include=['float64']).copy()
integers = data.select_dtypes(include=['int64']).copy()
dates = data.select_dtypes(include=['datetime64[ns]']).copy()
objects = data.select_dtypes(include=['object']).copy()
booleans = data.select_dtypes(include=['bool']).copy()

# FILLING NULL NUMERIC VALUES
if len(numerics.columns.values) > 0:
    numerics.fillna(value=0.0, inplace=True)
elif len(numerics.columns.values) == 0:
    exit
# FILLING NULL INTEGER VALUES    
if len(integers.columns.values) > 0:
    integers.fillna(value=0, inplace=True)
elif len(integers.columns.values) == 0:
    exit
    
# COMBINING NUMERIC(FLOAT) AND INTEGER VARIABLES INTO A COMBINED 'NUMBER' DF
numbers = pd.concat([numerics, integers], axis=1, sort=False)

# PRINTING COUNTS ASSOCIATED WITH OBJECTS AND CATEGORIES  
#for column in objects.columns:
#    display(objects[column].value_counts())
#    print('\n')

# CREATING CODES FOR OBJECT AND CATEGORY VARIABLES - USEFUL FOR STRING OPERATIONS
for obj in objects.columns:
    colName = str(obj) + 'code'
    objects[obj] = objects[obj].astype('category')
    objects[colName] = objects[obj].cat.codes

In [ ]:
# CREATING ALTERNATE SUBSETS OF THE PRIMARY DATAFRAME BASED ON SPECIFIC VARIABLES (EX. DATES OR CATEGORIES)
# THIS CODE SHOULD BE CLEANED UP - THE SYNTAX CURRENTLY ONLY ALLOWS FOUR FILTER-VARIABLES.

variables = list(data.columns.values)

print(variables, '\n')

print('DO YOU NEED TO FILTER BY SPECIFIC VARIABLES?')
confirmation = input('ENTER y or n: ')
filters = input('ENTER THE VARIABLES AS A COMMA SEPARATED LIST: ')
# SPLITTING THE VARIABLE LIST BY COMMAS
# STRIPPING POSSIBLE WHITE SPACE BETWEEN LIST ELEMENTS
filter_list = [x.strip() for x in filters.split(',')]

additional_var_num = len(filter_list)
additional_var_num = int(additional_var_num)

# SELECTING THE ARITHMETIC GROUPING METHOD
grouping_method = input('HOW DO YOU WANT TO CONSOLIDATE THE DATA - SUM OR AVERAGE? ')



if (confirmation == 'y') and (grouping_method == 'sum' or 'SUM' or 'Sum'):
    # CREATING ALTERNATE DATAFRAMES AROUND EACH OF THE SPECIFIED COLUMNS
    for var in filter_list:
        var = str(var)
        data_b = pd.DataFrame(data=data.groupby(by=[filter_list[0]], as_index=False).sum())
        filter_list.remove(var)
    for var in filter_list:
        length = len(filter_list)
        data_c = pd.DataFrame(data=data.groupby(by=[filter_list[0]], as_index=False).sum())
        filter_list.remove(var)
        if len(filter_list) > 0:
            for var in filter_list:
                data_d = pd.DataFrame(data=data.groupby(by=[filter_list[0]], as_index=False).sum())
                filter_list.remove(var)
            for var in filter_list:
                data_e = pd.DataFrame(data=data.groupby(by=[filter_list[0]], as_index=False).sum())
                filter_list.remove(var)
        elif len(filter_list) == 0:
            break
    if (confirmation == 'y') and grouping_method == 'average' or 'AVERAGE' or 'Average':
        for var in filter_list:
            var = str(var)
            data_b = pd.DataFrame(data=data.groupby(by=[filter_list[0]], as_index=False).mean())
            filter_list.remove(var)
        for var in filter_list:
            length = len(filter_list)
            data_c = pd.DataFrame(data=data.groupby(by=[filter_list[0]], as_index=False).mean())
            filter_list.remove(var)
            if len(filter_list) > 0:
                for var in filter_list:
                    data_d = pd.DataFrame(data=data.groupby(by=[filter_list[0]], as_index=False).mean())
                    filter_list.remove(var)
                for var in filter_list:
                    data_e = pd.DataFrame(data=data.groupby(by=[filter_list[0]], as_index=False).mean())
                    filter_list.remove(var)
            elif len(filter_list) == 0:
                break
                
elif confirmation == 'n':
    exit()
    
# PRINTING THE HEAD OF THE ALTERNATE DATAFRAMES
if additional_var_num == 1:
    print('\n', 'data_b')
    display(data_b.head())
elif additional_var_num <= 2:
    print('\n', 'data_b')
    display(data_b.head())
    print('\n', 'data_c')
    display(data_c.head())
elif additional_var_num <= 3:
    print('\n', 'data_b')
    display(data_b.head())
    print('\n', 'data_c')
    display(data_c.head())
    print('\n', 'data_d')
    display(data_d.head())
elif additional_var_num <= 4:
    print('\n', 'data_b')
    display(data_b.head())
    print('\n', 'data_c')
    display(data_c.head())
    print('\n', 'data_d')
    display(data_d.head())
    print('\n', 'data_e')
    display(data_e.head())

Typically, unique identifiers link entities across complex datasets. This is especially relevant to financial services, where various clients are linked to specific funds, holdings/securities, and other clients. With that said, unique IDs are used across industries: supply chains,retail, merchandising, broad tech, etc.

The code below extracts naming conventions for clients, and the associated unique identifiers, and creates aligned observations of each unique identifier related to a specific client. The code would need to be altered depending on the dataset.

This is certainly not relevant to every data science position, so the code will be commented out; however, if you need it...

In [ ]:
# -------------------------------------------------------------------
# Extracting columns that contain unique identifiers
# -------------------------------------------------------------------

#identifiers = list(variables[variables.str.contains('ID') == True])
#identifiers_o = list(variables[variables.str.contains('id') == True])
#identifiers_t = list(variables[variables.str.contains('Id') == True])
#identifiers_th = list(variables[variables.str.contains('i.d.') == True])
#identifiers_f = list(variables[variables.str.contains('I.D.') == True])
#client_naming = list(variables[variables.str.contains('name') == True])
#client_naming_alt = list(variables[variables.str.contains('client') == True])
#client_naming_altO = list(variables[variables.str.contains('Client') == True])
#client_naming_altT = list(variables[variables.str.contains('CLIENT') == True])

#mapping_cols = [identifiers, identifiers_o, identifiers_t, identifiers_th, identifiers_f, 
#                client_naming, client_naming_alt, client_naming_altO, client_naming_altT,]

#mapping_fields = []

#for col in mapping_cols:
#    if len(col) > 0:
#        mapping_fields.append(col); 

#identifiers = []

#for identifier in mapping_fields:
#    identifiers.append(identifier);

#vector_list = []

#for identifier in identifiers:
#    series = data[identifier]
#    vector_list.append(series)

#identifying_data = pd.concat(objs=vector_list, axis=1)
#print(identifying_data.head(), '\n')
        

#historical_mapping = pd.read_excel('H:\\Application_Data\\MAIN\\mapping\\identifier_archive\\mapping.xlsx', header=0)

#mapping_zip = pd.merge(left=historical_mapping, right=identifying_data, left_on='Client Code', right_on='')


# HERE WE ARE SAVING THE MAPPING FILE TO A SPECIFIC DRIVE, AS AN EXCEL FILE. 

#with pd.ExcelWriter('H:\\Application_Data\\MAIN\\mapping\\identifier_archive\\mapping.xlsx') as writer:  # doctest: +SKIP
#     mapping_zip.to_excel(writer, sheet_name='Sheet_name_1', index=False)

Visualization is an important part of data science. Graphs can elucidate the relationships between variables and uncover trends, which often leads to trajectory-altering decisions.

The code below contains simple loops that allow you to quickly produce charts for the columns (/series) of a particular dataset.  

These functions produce 'bare-bones' charts. Plotly and Matplotlib have the ability to produce beautiful visualizations - 

Plotly Documentation
https://plotly.com/python/

Matplotlib Documentation
https://matplotlib.org/

In [ ]:
# THE FUNCTION BELOW PLOTS HISTOGRAMS (/ PROBABILITY DENSITY FUNCTIONS) FOR NUMBERIC AND INTEGER VARIABLES.
# 
# THE OUTPUT IS A GAUSSIAN NORMAL DISTRIBUTION USING ACCURATE CALCULATIONS OF AVERAGE AND STANDARD DEVIATION.
# -------------------------------------------------------------------------------------------------------------

def gauss_dist(x):
    # The input is/should be a series
    # Set the style 
    sns.set(style='darkgrid')

    # Define the measures

    average = x.mean()
    standardDev = x.std()

    s = np.random.normal(average, standardDev, 1000)
    bins = 30
    plt.hist(s, 30)
    plt.plot(bins, 1/(standardDev * np.sqrt(2 * np.pi)) *
             np.exp( - (bins - average)**2 / (2 * standardDev**2)),
             linewidth=2, color='r')

    plt.title(str(x.name))

    plt.show()
    
    
# -----------------------------------------------------------------------------------------------       
# CALLING THE GAUSSIAN DISTRIBUTION FUNCTION ON THE SUBSET OF 'NUMBER' VARIABLES
# -----------------------------------------------------------------------------------------------

if len(numbers.columns.values) > 0:
    for column in numbers:
        viz_data = pd.Series(data=numbers[column], index=None)
        if viz_data.isnull().sum() == len(viz_data):
            break
        elif viz_data.isnull().sum() != len(viz_data):
            gauss_dist(viz_data);
        elif len(numbers.columns.values) == 0:
            break

In [ ]:
# SEABORN (SNS) PROVIDES A VERY GOOD PREBUILT DISTRIBUTION PLOT FUNCTION
#
# THE FUNCTION BELOW PRODUCES A UNIVARIATE DISTRIBUTION
# -------------------------------------------------------------------------

def sns_dist(x):
    sns.set(color_codes=True)
    sns.distplot(x)
    plt.show()
    
# THE FUNCTION BELOW PRODUCES AN ALTERNATE HISTOGRAM WITH A RUG CHART USING THE SEABORN LIBRARY

def sns_hist(x):
    sns.set(color_codes=True)
    sns.distplot(x, kde=False, rug=True)
    plt.show()
    
# -----------------------------------------------------------------------------------------------       
# CALLING THE SNS DISTRIBUTION FUNCTION ON THE SUBSET OF 'NUMBER' VARIABLES
# -----------------------------------------------------------------------------------------------

if len(numbers.columns.values) > 0:
    for column in numbers:
        viz_data = pd.Series(data=numbers[column], index=None)
        if viz_data.isnull().sum() == len(viz_data):
            break
        elif viz_data.isnull().sum() != len(viz_data):
            sns_dist(viz_data);
        elif len(numbers.columns.values) == 0:
            break
            
if len(numbers.columns.values) > 0:
    for column in numbers:
        viz_data = pd.Series(data=numbers[column], index=None)
        if viz_data.isnull().sum() == len(viz_data):
            break
        elif viz_data.isnull().sum() != len(viz_data):
            sns_hist(viz_data);
        elif len(numbers.columns.values) == 0:
            break

In [ ]:
# CONTINUOUS, AND DISCRETE VARIABLES, TYPICALLY DEVIATE ACROSS TIME.
# THIS FUNCTION PLOTS A SPECIFIED y VARIABLE AGAINST A SPECIFIED x VARIABLE (USUALLY DATE/TIME).
# WE CALCULATE THE STANDARD DEVIATION AND CREATE UPPER AND LOWER BOUNDS OF VARIANCE.
# ---------------------------------------------------------------------------------------------

def deviation_plot (data):
    
    print(data.columns.values)

    x_label = input('Enter the x-axis variable: ')
    y_label = input('Enter the y-axis variable: ')

    x = data[x_label]
    y = data[y_label]

    y_dev = y.std()


    upper_bound = go.Scatter(
        name='Upper Bound',
        x=x,
        y=y+y_dev,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty')

    trace = go.Scatter(
        name='Trend',
        x=x,
        y=y,
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty')

    lower_bound = go.Scatter(
        name='Lower Bound',
        x=x,
        y=y-y_dev,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines')

    # Trace order can be important
    # with continuous error bars
    data = [lower_bound, trace, upper_bound]

    layout = go.Layout(
        yaxis=dict(title=str(y_label)),
        title=(str(y_label)+' '+'against'+' '+str(x_label)),
        showlegend = False)

    fig = go.Figure(data=data, layout=layout)
    #py.iplot(fig, filename='pandas-continuous-error-bars')
    fig.show()
    

# CALLING THE FUNCTION ON THE DATE SUBSET (data_b)
deviation_plot(data_b)

In [ ]:
# THE RELATIONSHIP BETWEEN BINARY VARIABLES LIKE SEX (M/F), AND OTHER CATEGORICAL VARIABLES, CAN USUALLY BE
# CLEARLY ILLUSTRATED BY PLOTTING A HORIZONTAL BAR CHART. USING THE y-AXIS FOR THESE VARIABLES IS OFTEN IDEAL.
#
# USING THE TEST DATA ONE MIGHT LIKE TO COMPARE TOTAL DEATHS TO TOTAL CASES ACROSS LOCATIONS. THE FUNCTION BELOW
# WAS CRAFTED TO DO SO. 
# ---------------------------------------------------------------------------------------------------------------

def horizontal_bar (data):
    
    print(data.columns.values)
    
    x_label = input('Enter the x variable: ')
    xt_label = input('Enter a comparative x variable: ')
    y_label = input('Enter the y variable: ')

    x = data[x_label]
    xt = data[xt_label]
    y = data[y_label]
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=y,
        x=x,
        name=(str(x_label)),
        orientation='h',
        marker=dict(
            color='rgba(246, 78, 139, 0.6)',
            line=dict(color='rgba(246, 78, 139, 1.0)')
        )
    ))
    fig.add_trace(go.Bar(
        y=y,
        x=xt,
        name=(str(xt_label)),
        orientation='h',
        marker=dict(
            color='rgba(58, 71, 80, 0.6)',
            line=dict(color='rgba(58, 71, 80, 1.0)')
        )
    ))

    fig.update_layout(barmode='stack')
    fig.show()
    
    
    
# CALLING THE FUNCTION ON THE LOCATION SUBSET (data_c)
horizontal_bar(data_c)

# --------------------------------------------------------------------------------------
# AS MENTIONED, EXCEPTIONAL ANALYTICAL WORK REQUIRES REWORKING AND RESHAPING THE DATA
# 
# AS YOU CAN SEE, THE FIRST PLOT IS VERY CROWDED. CERTAIN COUNTRIES HAVE BEEN AFFECTED BY 
# COVID MORE THAN OTHERS. WE WILL SUBSET 'data_c' BASED ON THE AVERAGES OF EACH VARIABLE

ncase_mean = np.mean(data_c['new_cases'])
ndeath_mean = np.mean(data_c['new_deaths'])
tcase_mean = np.mean(data_c['total_cases'])
tdeath_mean = np.mean(data_c['total_deaths'])

# THERE ARE MORE EFFICIENT WAYS TO DO THIS - USING FEWER LINES OF CODE - 
# BUT, IF YOU ARE UNFAMILIAR, SEEING THE INDIVIDUAL FUNCTIONS IN ACTION WILL HELP

new_cases = data_c[data_c['new_cases']>=ncase_mean]
new_cases = new_cases.drop(labels=['new_deaths', 'total_cases', 'total_deaths'], axis=1)

new_deaths = data_c[data_c['new_deaths']>=ndeath_mean]
new_deaths = new_deaths.drop(labels=['new_cases', 'total_cases', 'total_deaths'], axis=1)

total_cases = data_c[data_c['total_cases']>=tcase_mean]
total_cases = total_cases.drop(labels=['new_cases', 'new_deaths','total_deaths'], axis=1)

total_deaths = data_c[data_c['total_deaths']>=tdeath_mean]
total_deaths = total_deaths.drop(labels=['new_cases', 'new_deaths','total_cases'], axis=1)

# EACH OF THESE DATAFRAMES WERE FILTERED BASED ON THE SPECIFIC AVERAGE
# THE OTHER COLUMNS WERE DROPPED. 
# NOW, WE WILL MERGE THEM BACK TOGETHER USING TWO DIFFERENT JOIN-TYPES - INNER AND OUTER

affected = new_cases.merge(new_deaths, how='inner', on='location')
affected = affected.merge(total_cases, how='inner', on='location')
affected = affected.merge(total_deaths, how='inner', on='location')

affected_alt = new_cases.merge(new_deaths, how='outer', on='location')
affected_alt = affected_alt.merge(total_cases, how='outer', on='location')
affected_alt = affected_alt.merge(total_deaths, how='outer', on='location')

affected_alt.fillna(method='bfill', inplace=True)

print('THIS DATA SET ILLUSTRATES A SPECIFIC TIME PERIOD: ', data['date'].min(), '-', data['date'].max(), '\n')

# CALLING THE HORIZONTAL BAR VISUAL FUNCTION ON THE DATASETS OF 'HIGHLY AFFECTED COUNTRIES'

horizontal_bar(affected)

# VISIBLE INACCURACIES - BASED ON THE OUTER JOIN WHEN STACKING TOTAL_CASES & TOTAL_DEATHS
horizontal_bar(affected_alt)


In [ ]:
# THIS IS A USEFUL FUNCTION TO CREATE A SCATTER, REGRESSION PLOT, AND JOINTGRID
# HOWEVER, BASED ON MY KNOWLEDGE, JOINTGRID WILL SOON BE DEPRECATED.

def scatter_reg_plot (x, y):

    # The two function inputs should be Series of the same length
    x = pd.Series(data=x, index=None)
    y = pd.Series(data=y, index=None)


    sns.set(style='darkgrid', palette='deep')
    
    plt.scatter(x=x, y=y, marker='o', alpha=0.5)

    plt.show()
    
    sns.regplot(x=x, y=y, marker="+", data=data, ci=95)
    
    plt.show()

    #R^2 FUNCTION
    def r2(x, y):
        return stats.pearsonr(x, y)[0] ** 2

    sns.jointplot(x=x, y=y, kind="reg", stat_func=r2)
    plt.show()
    
    
# CALLING THE FUNCTIONS USING TWO VARIABLES FROM THE 'NUMBERS' SUBSET
scatter_reg_plot(numbers['total_deaths'], numbers['total_cases'])

[Link to our Patreon Page]("https://www.patreon.com/think_elucidate")

In [ ]:
from IPython.display import clear_output
IPython.display.clear_output(wait=False)